In [63]:
import pandas as pd
import pyreadstat
from functools import reduce
pd.set_option('display.max_rows', None)

In [ ]:
def load_xpt(file_path, columns):

    data, _ = pyreadstat.read_xport(file_path)  
    return data[columns] 


file_paths = {
        "body_measures": "../data/raw/2017-2020/P_BMX.xpt",
        "demographics": "../data/raw/2017-2020/P_DEMO.xpt",
        "blood_pressure": "../data/raw/2017-2020/P_BPXO.xpt",
        "LabData_fast_glucose": "../data/raw/2017-2020/P_GLU.xpt",
        "LabData_glycohemog": "../data/raw/2017-2020/P_GHB.xpt",
        "LabData_insulin": "../data/raw/2017-2020/P_INS.xpt",
        "LabData_Cholest_total": "../data/raw/2017-2020/P_TCHOL.xpt",
        "LabData_Cholest_HDL": "../data/raw/2017-2020/P_HDL.xpt",
        "LabData_Cholest_LDL": "../data/raw/2017-2020/P_TRIGLY.xpt",
        "LabData_C_protein": "../data/raw/2017-2020/P_HSCRP.xpt",
        "LabData_Feretin": "../data/raw/2017-2020/P_FERTIN.xpt",
        "SurveyData_Alcohol": "../data/raw/2017-2020/P_ALQ.xpt",
        "SurveyData_Pressure_Cholest": "../data/raw/2017-2020/P_BPQ.xpt",
        "SurveyData_Diabetes": "../data/raw/2017-2020/P_DIQ.xpt",
        "SurveyData_Diet_Behavior": "../data/raw/2017-2020/P_DBQ.xpt",
        "SurveyData_Insurance": "../data/raw/2017-2020/P_HIQ.xpt",
        "SurveyData_Income": "../data/raw/2017-2020/P_INQ.xpt",
        "SurveyData_Medical": "../data/raw/2017-2020/P_MCQ.xpt",
        "SurveyData_MentalHealth": "../data/raw/2017-2020/P_DPQ.xpt",
        "SurveyData_PhysicalActivity": "../data/raw/2017-2020/P_PAQ.xpt",
        "SurveyData_SleepDisorders": "../data/raw/2017-2020/P_SLQ.xpt",
        "SurveyData_Smoking": "../data/raw/2017-2020/P_SMQ.xpt",
        "SurveyData_WeightHistory": "../data/raw/2017-2020/P_WHQ.xpt"
            }



columns_to_keep = {
    "body_measures": ["SEQN", "BMXWT", "BMXHT", "BMXBMI", "BMXWAIST", "BMXHIP"],
    "demographics": ["SEQN", "SDDSRVYR", "RIAGENDR", "RIDAGEYR","RIDRETH3","DMDBORN4","DMDEDUC2","RIDEXPRG","INDFMPIR","DMDMARTZ","WTINTPRP","WTMECPRP"],
    "blood_pressure": ["SEQN", "BPXOSY1", "BPXOSY2", "BPXOSY3", "BPXODI1", "BPXODI2", "BPXODI3","BPXOPLS1", "BPXOPLS2", "BPXOPLS3"],
    "LabData_fast_glucose": ["SEQN","WTSAFPRP","LBDGLUSI"],
    "LabData_glycohemog": ["SEQN","LBXGH"],
    "LabData_insulin": ["SEQN","WTSAFPRP","LBXIN","LBDINSI"],      # compaire WTSAFPRP to glucose
    "LabData_Cholest_total": ["SEQN","LBXTC","LBDTCSI"],
    "LabData_Cholest_HDL": ["SEQN","LBDHDD","LBDHDDSI"],
    "LabData_Cholest_LDL": ["SEQN","WTSAFPRP","LBXTR", "LBDTRSI","LBDLDL","LBDLDLM","LBDLDLN"],         # compaire WTSAFPRP to glucose insulin
    "LabData_C_protein": ["SEQN","LBXHSCRP",], 
    "LabData_Feretin": ["SEQN","LBXFER","LBDFERSI"], 
    "SurveyData_Alcohol" : ["SEQN","ALQ111","ALQ121","ALQ130","ALQ142"],
    "SurveyData_Pressure_Cholest" : ["SEQN","BPQ020","BPQ040A","BPQ050A","BPQ080", "BPQ090D"],
    "SurveyData_Diabetes" : ["SEQN","DIQ160","DID250"],
    "SurveyData_Diet_Behavior" : ["SEQN","DBD900","DBD905","DBD910","DBQ197","DBD895"],
    "SurveyData_Insurance" : ["SEQN","HIQ011"],
    "SurveyData_Income" : ["SEQN","INDFMMPI","INDFMMPC"],
    "SurveyData_Medical" : ["SEQN","MCQ080","MCQ160B","MCQ160C","MCQ160M","MCQ300C","MCQ366B","MCQ550"],
    "SurveyData_MentalHealth" : ["SEQN","DPQ020","DPQ030","DPQ040","DPQ060","DPQ070"],
    "SurveyData_PhysicalActivity" : ["SEQN","PAQ605","PAQ620","PAQ635","PAQ650","PAQ665","PAD680"],
    "SurveyData_SleepDisorders" : ["SEQN","SLD012","SLD013","SLQ120"],
    "SurveyData_Smoking" : ["SEQN","SMQ020","SMD650"],
    "SurveyData_WeightHistory" : ["SEQN","WHD110","WHD120","WHD140"]

}

dataframes = {}

for key, path in file_paths.items():
    try:
        if key == "LabData_insulin":
            data, _ = pyreadstat.read_xport(path, encoding="latin1")
        else:
            data, _ = pyreadstat.read_xport(path)

        dataframes[key] = data[columns_to_keep[key]]
        print(f"The file '{key}' has been successfully loaded.")
    except Exception as e:
        print(f"Error loading the file '{key}': {e}")


#final_dataset = reduce(
#    lambda left, right: left.merge(right, on="SEQN", how="inner"), 
#    dataframes.values()


dataframes["LabData_insulin"].describe()

daata = dataframes["LabData_insulin"]
daata[daata["LBXIN"]>30]

#final_dataset.head()

The file 'body_measures' has been successfully loaded.
The file 'demographics' has been successfully loaded.
The file 'blood_pressure' has been successfully loaded.
The file 'LabData_fast_glucose' has been successfully loaded.
The file 'LabData_glycohemog' has been successfully loaded.
The file 'LabData_insulin' has been successfully loaded.
The file 'LabData_Cholest_total' has been successfully loaded.
The file 'LabData_Cholest_HDL' has been successfully loaded.
The file 'LabData_Cholest_LDL' has been successfully loaded.
The file 'LabData_C_protein' has been successfully loaded.
The file 'LabData_Feretin' has been successfully loaded.
The file 'SurveyData_Alcohol' has been successfully loaded.
The file 'SurveyData_Pressure_Cholest' has been successfully loaded.
The file 'SurveyData_Diabetes' has been successfully loaded.
The file 'SurveyData_Diet_Behavior' has been successfully loaded.
The file 'SurveyData_Insurance' has been successfully loaded.
The file 'SurveyData_Income' has been

,SEQN,WTSAFPRP,LBXIN,LBDINSI
23,109340.0,39804.558150,56.69,340.14
39,109380.0,0.000000,70.07,420.42
40,109382.0,20329.155055,30.99,185.94
47,109415.0,36787.925395,31.89,191.34
56,109454.0,29980.659949,76.36,458.16
...,...,...,...,...
4980,124519.0,25152.313454,35.54,213.24
4984,124533.0,77435.688475,33.82,202.92
4999,124572.0,41136.270609,32.97,197.82
5013,124616.0,27584.821736,32.60,195.60


In [41]:
dataframes["body_measures"].shape
dataframes["LabData_fast_glucose"].head(200)

,SEQN,WTSAFPRP,LBDGLUSI
0,109264.0,27533.174559,5.38
1,109271.0,18100.965319,5.72
2,109274.0,16281.758327,8.55
3,109277.0,32230.046209,5.11
4,109282.0,79007.100787,5.27
...,...,...,...
195,109918.0,14421.157609,6.22
196,109919.0,31497.464044,5.83
197,109931.0,62755.975598,5.94
198,109939.0,37463.006441,5.38


In [66]:
final_dataset = reduce(
    lambda left, right: left.merge(right, on="SEQN", how="outer"), 
    dataframes.values())

final_dataset.head()
final_dataset.shape

(15560, 90)

In [57]:
final_dataset.columns

Index(['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXHIP', 'SDDSRVYR',
       'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDBORN4', 'DMDEDUC2', 'RIDEXPRG',
       'INDFMPIR', 'DMDMARTZ', 'WTINTPRP', 'WTMECPRP', 'BPXOSY1', 'BPXOSY2',
       'BPXOSY3', 'BPXODI1', 'BPXODI2', 'BPXODI3', 'BPXOPLS1', 'BPXOPLS2',
       'BPXOPLS3', 'WTSAFPRP_x', 'LBDGLUSI', 'LBXGH', 'WTSAFPRP_y', 'LBXIN',
       'LBDINSI', 'LBXTC', 'LBDTCSI', 'LBDHDD', 'LBDHDDSI', 'WTSAFPRP',
       'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLM', 'LBDLDLN', 'LBXHSCRP',
       'LBXFER', 'LBDFERSI', 'ALQ111', 'ALQ121', 'ALQ130', 'ALQ142', 'BPQ020',
       'BPQ040A', 'BPQ050A', 'BPQ080', 'BPQ090D', 'DIQ160', 'DID250', 'DBD900',
       'DBD905', 'DBD910', 'DBQ197', 'DBD895', 'HIQ011', 'INDFMMPI',
       'INDFMMPC', 'MCQ080', 'MCQ160B', 'MCQ160C', 'MCQ160M', 'MCQ300C',
       'MCQ366B', 'MCQ550', 'DPQ020', 'DPQ030', 'DPQ040', 'DPQ060', 'DPQ070',
       'PAQ605', 'PAQ620', 'PAQ635', 'PAQ650', 'PAQ665', 'PAD680', 'SLD012',
       'SLD013',

In [58]:
# Извлекаем все признаки из columns_to_keep
expected_columns = set()
for columns in columns_to_keep.values():
    expected_columns.update(columns)

# Проверяем соответствие с итоговым набором
actual_columns = set(final_dataset.columns)

# Найти отсутствующие в итоговом датасете
missing_columns = expected_columns - actual_columns
extra_columns = actual_columns - expected_columns

print(f"Отсутствующие колонки: {missing_columns}")
print(f"Лишние колонки: {extra_columns}")


Отсутствующие колонки: set()
Лишние колонки: {'WTSAFPRP_x', 'WTSAFPRP_y'}


In [62]:
final_dataset.shape

(4438, 90)

In [67]:
for key, dataframe in dataframes.items():
    print(f"Датасет '{key}' содержит {len(dataframe)} строк.")

Датасет 'body_measures' содержит 14300 строк.
Датасет 'demographics' содержит 15560 строк.
Датасет 'blood_pressure' содержит 11656 строк.
Датасет 'LabData_fast_glucose' содержит 5090 строк.
Датасет 'LabData_glycohemog' содержит 10409 строк.
Датасет 'LabData_insulin' содержит 5090 строк.
Датасет 'LabData_Cholest_total' содержит 12198 строк.
Датасет 'LabData_Cholest_HDL' содержит 12198 строк.
Датасет 'LabData_Cholest_LDL' содержит 5090 строк.
Датасет 'LabData_C_protein' содержит 13772 строк.
Датасет 'LabData_Feretin' содержит 11983 строк.
Датасет 'SurveyData_Alcohol' содержит 8965 строк.
Датасет 'SurveyData_Pressure_Cholest' содержит 10195 строк.
Датасет 'SurveyData_Diabetes' содержит 14986 строк.
Датасет 'SurveyData_Diet_Behavior' содержит 15560 строк.
Датасет 'SurveyData_Insurance' содержит 15560 строк.
Датасет 'SurveyData_Income' содержит 15560 строк.
Датасет 'SurveyData_Medical' содержит 14986 строк.
Датасет 'SurveyData_MentalHealth' содержит 8965 строк.
Датасет 'SurveyData_PhysicalA

In [64]:
final_dataset.isna().sum()

SEQN             0
BMXWT           78
BMXHT           83
BMXBMI          90
BMXWAIST       237
BMXHIP         234
SDDSRVYR         0
RIAGENDR         0
RIDAGEYR         0
RIDRETH3         0
DMDBORN4         0
DMDEDUC2       186
RIDEXPRG      3568
INDFMPIR       630
DMDMARTZ       186
WTINTPRP         0
WTMECPRP         0
BPXOSY1        455
BPXOSY2        459
BPXOSY3        470
BPXODI1        455
BPXODI2        459
BPXODI3        470
BPXOPLS1       758
BPXOPLS2       762
BPXOPLS3       771
WTSAFPRP_x       0
LBDGLUSI       265
LBXGH          234
WTSAFPRP_y       0
LBXIN          359
LBDINSI        359
LBXTC          327
LBDTCSI        327
LBDHDD         327
LBDHDDSI       327
WTSAFPRP         0
LBXTR          338
LBDTRSI        338
LBDLDL         370
LBDLDLM        370
LBDLDLN        346
LBXHSCRP       355
LBXFER         279
LBDFERSI       279
ALQ111         273
ALQ121         679
ALQ130        1532
ALQ142        1532
BPQ020           0
BPQ040A       2762
BPQ050A       2909
BPQ080      

In [81]:
from functools import reduce

# Объединяем все таблицы с помощью outer join
full_dataset = reduce(
    lambda left, right: left.merge(right, on="SEQN", how="outer"),
    dataframes.values()
)

# Фильтруем строки, где ключевые признаки не равны 0 (например, LBXGH и LBDGLUSI)
filtered_dataset = full_dataset[
    full_dataset["LBXGH"].notna() & (full_dataset["LBXGH"] != 0) &
    full_dataset["LBDGLUSI"].notna() & (full_dataset["LBDGLUSI"] != 0)
]

# Результат

#print(f"Итоговый датасет после фильтрации содержит {filtered_dataset.shape[0]} строк и {filtered_dataset.shape[1]} колонок.")

In [82]:
full_dataset.head()
filtered_dataset.shape

(4732, 90)

In [83]:
filtered_dataset.isna().sum()

SEQN             0
BMXWT           73
BMXHT           79
BMXBMI          85
BMXWAIST       208
BMXHIP         205
SDDSRVYR         0
RIAGENDR         0
RIDAGEYR         0
RIDRETH3         0
DMDBORN4         0
DMDEDUC2       739
RIDEXPRG      3922
INDFMPIR       634
DMDMARTZ       739
WTINTPRP         0
WTMECPRP         0
BPXOSY1        468
BPXOSY2        472
BPXOSY3        484
BPXODI1        468
BPXODI2        472
BPXODI3        484
BPXOPLS1       783
BPXOPLS2       787
BPXOPLS3       797
WTSAFPRP_x       0
LBDGLUSI         0
LBXGH            0
WTSAFPRP_y       0
LBXIN          116
LBDINSI        116
LBXTC           92
LBDTCSI         92
LBDHDD          92
LBDHDDSI        92
WTSAFPRP         0
LBXTR           92
LBDTRSI         92
LBDLDL         125
LBDLDLM        125
LBDLDLN        100
LBXHSCRP       124
LBXFER          31
LBDFERSI        31
ALQ111         811
ALQ121        1184
ALQ130        1992
ALQ142        1992
BPQ020         376
BPQ040A       3162
BPQ050A       3302
BPQ080      

In [85]:
filtered_dataset['LBXGH'].describe()

count    4732.000000
mean        5.789413
std         1.104622
min         2.800000
25%         5.300000
50%         5.500000
75%         5.900000
max        15.400000
Name: LBXGH, dtype: float64

In [111]:
import os
import pyreadstat

def load_data(base_path, file_paths, columns_to_keep):
    """
    Загружает данные из указанного периода.
    """
    dataframes = {}
    for key, relative_path in file_paths.items():
        # Construct the full file path
        file_path = os.path.join(base_path, relative_path)
        try:
            # Check if the file exists
            if not os.path.exists(file_path):
                print(f"Error: File '{file_path}' does not exist!")
                continue

            # Read the file with special handling for LabData_insulin
            if key == "LabData_insulin":
                data, _ = pyreadstat.read_xport(file_path, encoding="latin1")
            else:
                data, _ = pyreadstat.read_xport(file_path)

            # Filter columns based on the specified list
            dataframes[key] = data[columns_to_keep[key]]
            print(f"The file '{key}' has been successfully loaded.")
        except Exception as e:
            print(f"Error loading the file '{key}': {e}")
    return dataframes

# Paths for different periods
base_paths = {
    "2017-2020": "../data/raw/2017-2020",
    "2015-2016": "../data/raw/2015-2016"
}

# Relative paths (common for all periods)
file_paths = {
    "body_measures": "P_BMX.xpt",
    "demographics": "P_DEMO.xpt",
    "blood_pressure": "P_BPXO.xpt",
    "LabData_fast_glucose": "P_GLU.xpt",
    "LabData_glycohemog": "P_GHB.xpt",
    "LabData_insulin": "P_INS.xpt",
    "LabData_Cholest_total": "P_TCHOL.xpt",
    "LabData_Cholest_HDL": "P_HDL.xpt",
    "LabData_Cholest_LDL": "P_TRIGLY.xpt",
    "LabData_C_protein": "P_HSCRP.xpt",
    "LabData_Feretin": "P_FERTIN.xpt",
    "SurveyData_Alcohol": "P_ALQ.xpt",
    "SurveyData_Pressure_Cholest": "P_BPQ.xpt",
    "SurveyData_Diabetes": "P_DIQ.xpt",
    "SurveyData_Diet_Behavior": "P_DBQ.xpt",
    "SurveyData_Insurance": "P_HIQ.xpt",
    "SurveyData_Income": "P_INQ.xpt",
    "SurveyData_Medical": "P_MCQ.xpt",
    "SurveyData_MentalHealth": "P_DPQ.xpt",
    "SurveyData_PhysicalActivity": "P_PAQ.xpt",
    "SurveyData_SleepDisorders": "P_SLQ.xpt",
    "SurveyData_Smoking": "P_SMQ.xpt",
    "SurveyData_WeightHistory": "P_WHQ.xpt"
}

columns_to_keep = {
    "body_measures": ["SEQN", "BMXWT", "BMXHT", "BMXBMI", "BMXWAIST", "BMXHIP"],
    "demographics": ["SEQN", "SDDSRVYR", "RIAGENDR", "RIDAGEYR","RIDRETH3","DMDBORN4","DMDEDUC2","RIDEXPRG","INDFMPIR","DMDMARTZ","WTINTPRP","WTMECPRP"],
    "blood_pressure": ["SEQN", "BPXOSY1", "BPXOSY2", "BPXOSY3", "BPXODI1", "BPXODI2", "BPXODI3","BPXOPLS1", "BPXOPLS2", "BPXOPLS3"],
    "LabData_fast_glucose": ["SEQN","WTSAFPRP","LBDGLUSI"],
    "LabData_glycohemog": ["SEQN","LBXGH"],
    "LabData_insulin": ["SEQN","WTSAFPRP","LBXIN","LBDINSI"],      # compaire WTSAFPRP to glucose
    "LabData_Cholest_total": ["SEQN","LBXTC","LBDTCSI"],
    "LabData_Cholest_HDL": ["SEQN","LBDHDD","LBDHDDSI"],
    "LabData_Cholest_LDL": ["SEQN","WTSAFPRP","LBXTR", "LBDTRSI","LBDLDL","LBDLDLM","LBDLDLN"],         # compaire WTSAFPRP to glucose insulin
    "LabData_C_protein": ["SEQN","LBXHSCRP",], 
    "LabData_Feretin": ["SEQN","LBXFER","LBDFERSI"], 
    "SurveyData_Alcohol" : ["SEQN","ALQ111","ALQ121","ALQ130","ALQ142"],
    "SurveyData_Pressure_Cholest" : ["SEQN","BPQ020","BPQ040A","BPQ050A","BPQ080", "BPQ090D"],
    "SurveyData_Diabetes" : ["SEQN","DIQ160","DID250"],
    "SurveyData_Diet_Behavior" : ["SEQN","DBD900","DBD905","DBD910","DBQ197","DBD895"],
    "SurveyData_Insurance" : ["SEQN","HIQ011"],
    "SurveyData_Income" : ["SEQN","INDFMMPI","INDFMMPC"],
    "SurveyData_Medical" : ["SEQN","MCQ080","MCQ160B","MCQ160C","MCQ160M","MCQ300C","MCQ366B","MCQ550"],
    "SurveyData_MentalHealth" : ["SEQN","DPQ020","DPQ030","DPQ040","DPQ060","DPQ070"],
    "SurveyData_PhysicalActivity" : ["SEQN","PAQ605","PAQ620","PAQ635","PAQ650","PAQ665","PAD680"],
    "SurveyData_SleepDisorders" : ["SEQN","SLD012","SLD013","SLQ120"],
    "SurveyData_Smoking" : ["SEQN","SMQ020","SMD650"],
    "SurveyData_WeightHistory" : ["SEQN","WHD110","WHD120","WHD140"]

}

# Загрузка данных для каждого периода
dataframes_by_period = {}
for period, base_path in base_paths.items():
    print(f"Loading data for the period: {period}")
    dataframes_by_period[period] = load_data(base_path, file_paths, columns_to_keep)

Loading data for the period: 2017-2020
The file 'body_measures' has been successfully loaded.
The file 'demographics' has been successfully loaded.
The file 'blood_pressure' has been successfully loaded.
The file 'LabData_fast_glucose' has been successfully loaded.
The file 'LabData_glycohemog' has been successfully loaded.
The file 'LabData_insulin' has been successfully loaded.
The file 'LabData_Cholest_total' has been successfully loaded.
The file 'LabData_Cholest_HDL' has been successfully loaded.
The file 'LabData_Cholest_LDL' has been successfully loaded.
The file 'LabData_C_protein' has been successfully loaded.
The file 'LabData_Feretin' has been successfully loaded.
The file 'SurveyData_Alcohol' has been successfully loaded.
The file 'SurveyData_Pressure_Cholest' has been successfully loaded.
The file 'SurveyData_Diabetes' has been successfully loaded.
The file 'SurveyData_Diet_Behavior' has been successfully loaded.
The file 'SurveyData_Insurance' has been successfully loaded

In [101]:
dataframes_by_period['2017-2020']['body_measures'].head()

,SEQN,BMXWT,BMXHT,BMXBMI,BMXWAIST,BMXHIP
0,109263.0,NaN,NaN,NaN,NaN,NaN
1,109264.0,42.2,154.7,17.6,63.8,85.0
2,109265.0,12.0,89.3,15.0,41.2,NaN
3,109266.0,97.1,160.2,37.8,117.9,126.1
4,109269.0,13.6,NaN,NaN,NaN,NaN


In [112]:
from functools import reduce
import pandas as pd

# Словарь для хранения объединённых данных по периодам
merged_data_by_period = {}

# Объединение данных внутри каждого периода
for period, dfs in dataframes_by_period.items():
    print(f"Combining data for the period: {period}")
    
    # Проверяем, что все DataFrame содержат ключ 'SEQN'
    dataframes_to_merge = [df for df in dfs.values() if 'SEQN' in df.columns]
    
    if not dataframes_to_merge:
        print(f"No dataframes to merge for the period {period}")
        continue

    # Объединяем все DataFrame для текущего периода по ключу 'SEQN'
    merged_data_by_period[period] = reduce(
        lambda left, right: pd.merge(left, right, on="SEQN", how="outer"),
        dataframes_to_merge
    )

# Пример: просмотр объединённого DataFrame для периода '2017-2020'
merged_data_by_period['2017-2020'].head()


print(merged_data_by_period['2017-2020'].shape)

Combining data for the period: 2017-2020
Combining data for the period: 2015-2016
No dataframes to merge for the period 2015-2016
(15560, 90)


In [105]:
final_dataset_2017_2020

,SEQN
